In [4]:
import pandas as pd
import mysql.connector

# Load the Titanic dataset
df = pd.read_csv('train.csv')

# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='rootroot',
    database='titanic_db'
)
cursor = conn.cursor()

# Create dataframes for the related tables
df_embarkation_port = df[['Embarked']].drop_duplicates().reset_index(drop=True)
df_embarkation_port['EmbarkationPortId'] = df_embarkation_port.index + 1
df_embarkation_port['PortName'] = df_embarkation_port['Embarked'].map({
    'C': 'Cherbourg', 'Q': 'Queenstown', 'S': 'Southampton'
})

# Ensure 'Unknown' embarkation port is not added
df_embarkation_port = df_embarkation_port.dropna().reset_index(drop=True)

print("Final Embarkation Port Table")
print(df_embarkation_port)

df_class = pd.DataFrame({'ClassId': [1, 2, 3], 'ClassName': ['1st', '2nd', '3rd']})

df_ticket = df[['Ticket', 'Fare']].drop_duplicates().reset_index(drop=True)
df_ticket['TicketId'] = df_ticket.index + 1
df_ticket = df_ticket.fillna({'Ticket': 'Unknown', 'Fare': 0})

# Remove 'Unknown' cabin entry and handle NaN values properly
df_cabin = df[['Cabin']].drop_duplicates().reset_index(drop=True)
df_cabin = df_cabin.dropna().reset_index(drop=True)
df_cabin['CabinId'] = df_cabin.index + 1
df_cabin['Cabin'] = df_cabin['Cabin'].apply(lambda x: x if len(str(x)) <= 50 else str(x)[:50])

print(f"Original Passenger Data: {len(df)}")
print(f"Unique Embarkation Ports: {len(df_embarkation_port)}")
print(f"Unique Tickets: {len(df_ticket)}")
print(f"Unique Cabins: {len(df_cabin)}")

# Create the passenger table with foreign keys
df_passenger = df.merge(df_ticket, on=['Ticket', 'Fare'], how='left') \
                .merge(df_cabin, on='Cabin', how='left') \
                .merge(df_embarkation_port, on='Embarked', how='left', suffixes=('', '_port'))

# Check the size of the passenger dataframe after merging
print(f"Merged Passenger Data: {len(df_passenger)}")

df_passenger['PassengerId'] = df_passenger.index + 1
df_passenger.rename(columns={'Pclass': 'ClassId'}, inplace=True)
df_passenger = df_passenger[['PassengerId', 'Name', 'Sex', 'Age', 'Survived', 'ClassId', 'TicketId', 'CabinId', 'EmbarkationPortId', 'SibSp', 'Parch']]

# Ensure 'Sex' column values are valid and fit within the defined length
df_passenger['Sex'] = df_passenger['Sex'].apply(lambda x: x if x in ['male', 'female'] else 'Unknown')

# Set missing Age values to NULL
df_passenger['Age'] = df_passenger['Age'].replace(0, pd.NA)

# Set missing values to NULL
df_passenger = df_passenger.fillna({
    'Name': pd.NA, 'Sex': pd.NA, 'Survived': pd.NA, 
    'ClassId': pd.NA, 'TicketId': pd.NA, 'CabinId': pd.NA, 
    'EmbarkationPortId': pd.NA, 'SibSp': pd.NA, 'Parch': pd.NA
})

# Print the size of the passenger dataframe after handling missing data
print(f"Final Passenger Data: {len(df_passenger)}")

# Debugging: Check for any missing EmbarkationPortId values
missing_embarkation_ports = df_passenger[~df_passenger['EmbarkationPortId'].isin(df_embarkation_port['EmbarkationPortId'])]
print(f"Missing EmbarkationPortId in Passenger Data: {missing_embarkation_ports}")

# Debugging: Print unique EmbarkationPortId values in Passenger table
print(f"Unique EmbarkationPortId values in Passenger table: {df_passenger['EmbarkationPortId'].unique()}")

# Insert data into MySQL tables
def insert_data(df, table, columns):
    for _, row in df.iterrows():
        row = [None if pd.isna(val) else val for val in row]
        cursor.execute(f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})", tuple(row))
        conn.commit()

# Insert data in the correct order
insert_data(df_embarkation_port[['EmbarkationPortId', 'Embarked', 'PortName']], 'EmbarkationPort', ['EmbarkationPortId', 'PortCode', 'PortName'])
insert_data(df_class, 'Class', ['ClassId', 'ClassName'])
insert_data(df_ticket[['TicketId', 'Ticket', 'Fare']], 'Ticket', ['TicketId', 'TicketNumber', 'Fare'])
insert_data(df_cabin[['CabinId', 'Cabin']], 'Cabin', ['CabinId', 'CabinNumber'])
insert_data(df_passenger, 'Passenger', ['PassengerId', 'Name', 'Sex', 'Age', 'Survived', 'ClassId', 'TicketId', 'CabinId', 'EmbarkationPortId', 'SibSp', 'Parch'])

# Close the connection
cursor.close()
conn.close()

Final Embarkation Port Table
  Embarked  EmbarkationPortId     PortName
0        S                  1  Southampton
1        C                  2    Cherbourg
2        Q                  3   Queenstown
Original Passenger Data: 891
Unique Embarkation Ports: 3
Unique Tickets: 682
Unique Cabins: 147
Merged Passenger Data: 891
Final Passenger Data: 891
Missing EmbarkationPortId in Passenger Data:      PassengerId                                       Name     Sex   Age  \
61            62                        Icard, Miss. Amelie  female  38.0   
829          830  Stone, Mrs. George Nelson (Martha Evelyn)  female  62.0   

     Survived  ClassId  TicketId  CabinId  EmbarkationPortId  SibSp  Parch  
61          1        1        61     13.0                NaN      0      0  
829         1        1        61     13.0                NaN      0      0  
Unique EmbarkationPortId values in Passenger table: [ 1.  2.  3. nan]


In [3]:
pip install pandas mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 24.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
